In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from utils import plot_images, train_valid_split
from metric import apk, mapk
import lightgbm as lgb

import faiss
import psutil
from lightfm import LightFM
from scipy import sparse
import catboost

In [2]:
dataset = '100'

transactions = pd.read_pickle(f"input/{dataset}/transactions_train.pkl")
users = pd.read_pickle(f"input/{dataset}/users.pkl")
items = pd.read_pickle(f"input/{dataset}/items.pkl")
transactions['day'] = (transactions['t_dat'].max() - transactions['t_dat']).dt.days


In [3]:
def create_candidates(transactions: pd.DataFrame, target_users: np.ndarray, week: int) -> pd.DataFrame:
    """
    transactions
        original transactions (user, item, week)
    target_users, week
        候補生成対象のユーザー
        weekで指定されている週の段階での情報のみから作られる
    """
    assert len(target_users) == len(set(target_users))

    def create_candidates_repurchase(
            prefix: str,
            transactions: pd.DataFrame,
            target_users: np.ndarray,
            week_start: int,
            week_end: int) -> pd.DataFrame:
        tr = transactions.query(
            "user in @target_users and @week_start <= week <= @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)

        gr_week = tr.groupby(['user', 'item'])['week'].min().reset_index(name='week')
        gr_volume = tr.groupby(['user', 'item']).size().reset_index(name='volume')

        gr_week['week_rank'] = gr_week.groupby('user')['week'].rank()
        gr_volume['volume_rank'] = gr_volume.groupby('user')['volume'].rank()

        candidates = gr_week.merge(gr_volume, on=['user', 'item'])
        candidates = candidates[['user', 'item', 'week_rank', 'volume_rank']].rename(
            columns={'week_rank': f'{prefix}_week_rank', 'volume_rank': f'{prefix}_volume_rank'})
        return candidates

    def create_candidates_popular(
            prefix: str,
            transactions: pd.DataFrame,
            target_users: np.ndarray,
            week_start: int,
            week_end: int) -> pd.DataFrame:
        NUM_POPULAR_ITEMS = 12
        tr = transactions.query("@week_start <= week <= @week_end")[['user', 'item']].drop_duplicates(ignore_index=True)
        popular_items = tr['item'].value_counts().index.values[:NUM_POPULAR_ITEMS]
        popular_items = pd.DataFrame({
            'item': popular_items,
            'rank': range(NUM_POPULAR_ITEMS),
            'crossjoinkey': 1,
        })

        candidates = pd.DataFrame({
            'user': target_users,
            'crossjoinkey': 1,
        })

        candidates = candidates.merge(popular_items, on='crossjoinkey').drop('crossjoinkey', axis=1)
        candidates = candidates.rename(columns={'rank': f'{prefix}_rank'})
        return candidates

    def create_candidates_cooc(
            prefix: str,
            transactions: pd.DataFrame,
            week_start: int,
            week_end: int,
            base_candidates: pd.DataFrame) -> pd.DataFrame:
        tr = transactions.query("@week_start <= week == @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)
        tr = tr.merge(tr.rename(columns={'item': 'item_with', 'week': 'week_with'}), on='user').query(
            "item != item_with and week <= week_with")[['item', 'item_with']].reset_index(drop=True)
        gr_sz = tr.groupby('item').size().reset_index(name='tot')
        gr_cnt = tr.groupby(['item', 'item_with']).size().reset_index(name='cnt')
        item2item = gr_cnt.merge(gr_sz, on='item')
        item2item['ratio'] = item2item['cnt'] / item2item['tot']
        item2item = item2item.query("ratio >= 0.030730").reset_index(drop=True)

        base_candidates_columns = [c for c in base_candidates.columns if c not in ['user', 'item', 'week']]
        replace = {c: f"{prefix}_{c}" for c in base_candidates_columns}
        candidates = base_candidates.merge(item2item, on='item').drop(
            ['item', 'cnt'], axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)
        candidates = candidates.rename(columns={'ratio': f'{prefix}_ratio', 'tot': f'{prefix}_tot'})
        return candidates

    def create_candidates_same_product_code(
            prefix: str,
            items: pd.DataFrame,
            base_candidates: pd.DataFrame) -> pd.DataFrame:
        item2item = items[['item', 'product_code']].merge(items[['item', 'product_code']].rename(
            {'item': 'item_with'}, axis=1), on='product_code')[['item', 'item_with']].query("item != item_with").reset_index(drop=True)

        base_candidates_columns = [c for c in base_candidates.columns if c not in ['user', 'item', 'week']]
        replace = {c: f"{prefix}_{c}" for c in base_candidates_columns}
        candidates = base_candidates.merge(
            item2item,
            on='item').drop(
            'item',
            axis=1).rename(
            columns={
                'item_with': 'item'}).rename(
                    columns=replace)
        return candidates

    def create_candidates_same_category_top(
            strategy: str,
            transactions: pd.DataFrame,
            users: pd.DataFrame,
            items: pd.DataFrame,
            base_candidates: pd.DataFrame,
            week_start: int,
            week_end: int,
            category: str,
            ) -> pd.DataFrame:
        users = users.copy()
        users['age_segment'] = users['age'] // 10
        tr = transactions.query("@week_start <= week <= @week_end")[['user', 'item']].drop_duplicates(ignore_index=True)
        tr = tr.merge(items[['item', category]], on='item').merge(users[['user', 'age_segment']], on='user')
        category_popular_items = tr.groupby(['item', 'age_segment', category]).size().reset_index(name='cnt')
        category_popular_items['segment_rank'] = category_popular_items.groupby(category)['cnt'].rank(ascending=False, method='dense')
        category_popular_items = category_popular_items.query('segment_rank <= 3')

        base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
        replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
        candidates = (base_candidates
                      .merge(users[['user', 'age_segment']], on='user')
                      .merge(items[['item', category]], on='item')
                      .drop('item', axis=1)
                      .merge(category_popular_items, on=['age_segment', category])
                      .rename(columns=replace))[['user', 'item', 'segment_rank']]

        # candidates['strategy'] = strategy
        return candidates

    candidates = [
        create_candidates_repurchase('repurchase', transactions, target_users, week, week + 40),
        create_candidates_popular('pop', transactions, target_users, week, week),
    ]
    cat = 'product_type_no'
    candidates_same_category_top = create_candidates_same_category_top(f'same_{cat}', transactions, users, items, candidates[0], week, week, cat)
    candidates.append(candidates_same_category_top)
    candidates.append(create_candidates_cooc('cooc', transactions, week, week + 12, candidates[0]))
    candidates.append(create_candidates_same_product_code('same_product_code', items, candidates[0]))

    volumes = np.array([len(c) for c in candidates])
    print(volumes, volumes / np.sum(volumes))

    return pd.concat(candidates).reset_index(drop=True)


In [4]:
# valid: week=1
# train: week=2..1+num_train_weeks
num_train_weeks = 3
candidates = []
for week in range(1, 2+num_train_weeks):
    target_users = transactions[transactions['week'] == week]['user'].unique()
    candidates.append(create_candidates(transactions, target_users, week+1))

[1443705  827808 1874926 3449125 8777527] [0.08817547 0.05055905 0.11451265 0.21065815 0.53609468]
[1560164  864228 2190118 4908354 9468051] [0.08215318 0.04550744 0.11532451 0.258458   0.49855686]
[1605657  909864 2106867 5007577 9934564] [0.0820698  0.0465058  0.1076881  0.25595183 0.50778447]
[ 1684869   963036  2249793  4950146 10432776] [0.08307779 0.04748553 0.11093315 0.24408258 0.51442096]


In [5]:
def merge_labels(candidates: pd.DataFrame, week: int) -> pd.DataFrame:
    """
    candidates
        (user, item, strategy, score)

    Return
    ------
    (user, item, y, ...)
    """
    labels = transactions[transactions['week'] == week][['user', 'item']].drop_duplicates(ignore_index=True)
    labels['y'] = 1
    original_positives = len(labels)
    labels = candidates.merge(labels, on=['user', 'item'], how='left')
    labels['y'] = labels['y'].fillna(0)
    remaining_positives = labels[['user', 'item', 'y']].drop_duplicates(ignore_index=True)['y'].sum()
    recall = remaining_positives / original_positives
    print(f"Recall: {recall}")
    return labels



In [6]:
for idx in range(len(candidates)):
    candidates[idx] = merge_labels(candidates[idx], 1+idx)


Recall: 0.10469849528372512
Recall: 0.10414637356851389
Recall: 0.10465233498828096
Recall: 0.09782029376263854


In [7]:
def get_query_group(df):
    def run_length_encoding(sequence):
        comp_seq_index, = np.concatenate(([True], sequence[1:] != sequence[:-1], [True])).nonzero()
        return sequence[comp_seq_index[:-1]], np.ediff1d(comp_seq_index)
    users = df['user'].values
    _, group = run_length_encoding(users)
    return list(group)

def drop_trivial_users(labels):
    """
    LightGBMのxendgcやlambdarankでは正例のみや負例のみのuserは学習に無意味なのと、メトリックの計算がおかしくなるので省く
    """
    bef = len(labels)
    df = labels[labels['user'].isin(labels[['user', 'y']].drop_duplicates().groupby(
        'user').size().reset_index(name='sz').query("sz==2").user)].reset_index(drop=True)
    aft = len(df)
    print(f"drop trivial queries: {bef} -> {aft}")
    return df


In [8]:
for idx in range(len(candidates)):
    candidates[idx]['week'] = 1+idx

valid_all = candidates[0].copy()

for idx in range(len(candidates)):
    candidates[idx] = drop_trivial_users(candidates[idx])


drop trivial queries: 16373091 -> 5704122
drop trivial queries: 18990915 -> 6823387
drop trivial queries: 19564529 -> 7000798
drop trivial queries: 20280620 -> 6936689


In [9]:
def attach_features(transactions: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame, candidates: pd.DataFrame, week: int) -> pd.DataFrame:
    """
    week: これを含めた以前の情報は使って良い
    """
    n_original = len(candidates)
    df = candidates.copy()

    # user static features
    user_features = ['FN', 'Active', 'age', 'club_member_status_idx', 'fashion_news_frequency_idx']
    df = df.merge(users[['user'] + user_features], on='user')

    # item static features
    item_features = [c for c in items.columns if c.endswith('idx')]
    df = df.merge(items[['item'] + item_features], on='item')

    # user dynamic features (transactions)
    num_weeks = 8
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end").groupby('user')[['price', 'sales_channel_id']].agg(['mean', 'std'])
    tmp.columns = ['user_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
    df = df.merge(tmp, on='user', how='left')

    # item dynamic features (transactions)
    num_weeks = 8
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end").groupby('item')[['price', 'sales_channel_id']].agg(['mean', 'std'])
    tmp.columns = ['item_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
    df = df.merge(tmp, on='item', how='left')

    # item dynamic features (user features)
    num_weeks = 8
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end").merge(users[['user', 'age']], on='user')
    tmp = tmp.groupby('item')['age'].agg(['mean', 'std'])
    tmp.columns = [f'age_{a}' for a in tmp.columns.to_flat_index()]
    df = df.merge(tmp, on='item', how='left')

    # item price diff features (transactions)
    week_end = week + 2
    tmp = transactions.query("@week <= week < @week_end").groupby(['item', 'week'])[['price']].agg(['mean', 'std'])
    tmp.columns = ['item_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
    tmp_this_week = (tmp
                     .query('week==@week')[['item_price_mean', 'item_price_std']]
                     .rename(columns={'item_price_mean': 'item_price_mean_this_week', 'item_price_std': 'item_price_std_this_week'}))
    tmp_last_week = (tmp
                     .query('week==@week+1')[['item_price_mean', 'item_price_std']]
                     .rename(columns={'item_price_mean': 'item_price_mean_last_week', 'item_price_std': 'item_price_std_last_week'}))
    tmp2 = tmp_this_week.merge(tmp_last_week, on='item')
    tmp2['item_price_mean_diff'] = tmp2['item_price_mean_this_week'] - tmp2['item_price_mean_last_week']
    tmp2['item_price_std_ratio'] = tmp2['item_price_std_this_week'] / tmp2['item_price_std_last_week']
    df = df.merge(tmp2, on='item', how='left')
    
    # item dynamic features (segment rank)
    num_weeks = 2
    week_end = week + num_weeks
    users_tmp = users.copy()
    users_tmp['age_segment'] = users_tmp['age'] // 3
    tmp = (transactions
           .query("@week <= week < @week_end")
           .merge(users_tmp[['user', 'age_segment']], on='user')
           .merge(items[['item', 'product_type_no_idx']], on='item'))
    tmp = tmp.groupby(['age_segment', 'product_type_no_idx', 'item']).size().reset_index(name='cnt')
    tmp['rank'] = tmp.groupby(['age_segment', 'product_type_no_idx'])['cnt'].rank(ascending=False, method='dense')
    tmp = tmp[['age_segment', 'item', 'rank', 'cnt']].set_index(['age_segment', 'item'])
    tmp.columns = [f'product_type_no_idx_segment_{a}' for a in tmp.columns.to_flat_index()]
    df = df.merge(users_tmp[['user', 'age_segment']], on='user').merge(tmp, on=['age_segment', 'item'], how='left')
    
    # item dynamic features (repurchase num)
    num_weeks = 4
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end")
    tmp = tmp.groupby(['item', 'user']).size().reset_index(name='cnt')
    tmp = tmp.groupby('item')['cnt'].mean().reset_index(name='item_repurchase_volume')
    df = df.merge(tmp, on='item', how='left')

    # item freshness features
    tmp = transactions.query("@week <= week").groupby('item')['week'].min().reset_index(name='item_week_min')
    tmp['item_week_min'] -= week
    df = df.merge(tmp, on='item', how='left')

    # item volume features
    num_weeks = 1
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end").groupby('item').size().reset_index(name='item_volume')
    df = df.merge(tmp, on='item', how='left')

    # # user freshness features
    # tmp = transactions.query("@week <= week").groupby('user')['week'].min().reset_index(name='user_week_min')
    # tmp['user_week_min'] -= week
    # df = df.merge(tmp, on='user', how='left')

    # # user volume features
    # num_weeks = 1
    # week_end = week + num_weeks
    # tmp = transactions.query("@week <= week < @week_end").groupby('user').size().reset_index(name='user_volume')
    # df = df.merge(tmp, on='user', how='left')

    # user-item freshness features
    tmp = transactions.query("@week <= week").groupby(['user', 'item'])['week'].min().reset_index(name='user_item_week_min')
    tmp['user_item_week_min'] -= week
    df = df.merge(tmp, on=['item', 'user'], how='left')

    # user-item volume features
    num_weeks = 1
    week_end = week + num_weeks
    tmp = transactions.query("@week <= week < @week_end").groupby(['user', 'item']).size().reset_index(name='user_item_volume')
    df = df.merge(tmp, on=['user', 'item'], how='left')

    assert len(df) == n_original
    return df


valid_all = attach_features(transactions, users, items, valid_all, 2)
for idx in range(len(candidates)):
    candidates[idx] = attach_features(transactions, users, items, candidates[idx], 2+idx)

In [ ]:
for idx in range(len(candidates)):
    candidates[idx]['group'] = candidates[idx]['week'].astype(str) + '_' + candidates[idx]['user'].astype(str)
    candidates[idx] = candidates[idx].sort_values(by='group').reset_index(drop=True)

valid = candidates[0]

def concat_train(candidates, begin, num):
    train = pd.concat([candidates[idx] for idx in range(begin, begin+num)])
    return train

train = concat_train(candidates, 1, num_train_weeks)

In [ ]:
feature_columns = [c for c in valid.columns if c not in ['y', 'week', 'group']]
print(feature_columns)


In [ ]:
cat_feature_values = [c for c in feature_columns if c.endswith('idx')]
cat_features = [feature_columns.index(c) for c in cat_feature_values]
cat_features


In [ ]:
train_pool = catboost.Pool(data=train[feature_columns], label=train['y'], group_id=train['group'], cat_features=cat_features)
valid_pool = catboost.Pool(data=valid[feature_columns], label=valid['y'], group_id=valid['group'], cat_features=cat_features)

In [14]:
params = {
    'loss_function': 'YetiRank',
    'use_best_model': True,
    'one_hot_max_size': 300,
}
model = catboost.CatBoost(params)
model.fit(train_pool, eval_set=valid_pool)

3:	test: 0.2563400	best: 0.2565061 (2)	total: 1m 18s	remaining: 5h 26m 35s
4:	test: 0.2564610	best: 0.2565061 (2)	total: 1m 38s	remaining: 5h 25m 38s
5:	test: 0.2624585	best: 0.2624585 (5)	total: 1m 58s	remaining: 5h 26m 34s
6:	test: 0.2614847	best: 0.2624585 (5)	total: 2m 17s	remaining: 5h 25m 40s
7:	test: 0.2687994	best: 0.2687994 (7)	total: 2m 37s	remaining: 5h 25m 20s
8:	test: 0.2737060	best: 0.2737060 (8)	total: 2m 56s	remaining: 5h 24m 29s
9:	test: 0.2736723	best: 0.2737060 (8)	total: 3m 16s	remaining: 5h 23m 53s
10:	test: 0.2803116	best: 0.2803116 (10)	total: 3m 36s	remaining: 5h 23m 47s
11:	test: 0.2813949	best: 0.2813949 (11)	total: 3m 55s	remaining: 5h 23m 21s
12:	test: 0.2937329	best: 0.2937329 (12)	total: 4m 15s	remaining: 5h 22m 49s
13:	test: 0.2944561	best: 0.2944561 (13)	total: 4m 34s	remaining: 5h 22m 37s
14:	test: 0.2945290	best: 0.2945290 (14)	total: 4m 54s	remaining: 5h 22m 14s
15:	test: 0.2947898	best: 0.2947898 (15)	total: 5m 14s	remaining: 5h 21m 51s
16:	test: 0.3

In [15]:
# recent_items = transactions.query("2 <= week <= 4").item.unique()
# valid_all = valid_all.query("item in @recent_items")


In [16]:
valid_all['pred'] = model.predict(valid_all[feature_columns])
pred = valid_all.groupby(['user', 'item'])['pred'].max().reset_index()


In [17]:
pred = pred.sort_values(by=['user', 'pred'], ascending=False).reset_index(
    drop=True).groupby('user')['item'].apply(lambda x: list(x)[:12]).reset_index()
pred


,user,item
0,80,"[2865, 42129, 104045, 103108, 67, 3091, 67522,..."
1,86,"[100228, 27905, 33868, 98606, 102472, 102473, ..."
2,107,"[2219, 61305, 61303, 13042, 61304, 53892, 8465..."
3,117,"[104045, 3091, 71107, 103108, 67522, 104072, 9..."
4,179,"[95216, 105180, 104986, 3510, 95217, 105179, 3..."
...,...,...
68979,1371868,"[93158, 37185, 93157, 51179, 93159, 17043, 104..."
68980,1371871,"[67522, 104045, 71107, 3091, 42626, 104072, 10..."
68981,1371879,"[103668, 103665, 103669, 103666, 98867, 103670..."
68982,1371937,"[59774, 59773, 59771, 67522, 59772, 71107, 103..."


In [18]:
gt = transactions[transactions['week'] == 1].groupby('user')['item'].apply(list).reset_index().rename(columns={'item': 'gt'})
merged = gt.merge(pred, on='user', how='left')
merged['item'] = merged['item'].fillna('').apply(list)
merged


,user,gt,item
0,80,[28967],"[2865, 42129, 104045, 103108, 67, 3091, 67522,..."
1,86,[87371],"[100228, 27905, 33868, 98606, 102472, 102473, ..."
2,107,"[69711, 77256, 33872]","[2219, 61305, 61303, 13042, 61304, 53892, 8465..."
3,117,[97391],"[104045, 3091, 71107, 103108, 67522, 104072, 9..."
4,179,"[102397, 98409, 73, 95784, 103796, 105103, 103...","[95216, 105180, 104986, 3510, 95217, 105179, 3..."
...,...,...,...
68979,1371868,"[97531, 103424]","[93158, 37185, 93157, 51179, 93159, 17043, 104..."
68980,1371871,"[94310, 91533]","[67522, 104045, 71107, 3091, 42626, 104072, 10..."
68981,1371879,"[2118, 84994, 57078, 84991, 101099, 104036, 87...","[103668, 103665, 103669, 103666, 98867, 103670..."
68982,1371937,"[67261, 70640]","[59774, 59773, 59771, 67522, 59772, 71107, 103..."


In [19]:
mapk(merged['gt'], merged['item'])


0.03099711296249596

In [20]:
popular_items = transactions[transactions['week'] == 2]['item'].value_counts()[:12].index.tolist()
merged['item'] = merged['item'].apply(lambda x: x + popular_items)
mapk(merged['gt'], merged['item'])


0.03099711296249596

In [21]:
model.save_model('cat.txt')
# params = {
#     'loss_function': 'YetiRank',
#     'use_best_model': True,
#     'one_hot_max_size': 300,
# }

# model = catboost.CatBoost(params)
# model.load_model('cat.txt')

In [22]:
# assert False

submission

In [23]:
train = concat_train(candidates, 0, num_train_weeks)

In [ ]:
train_pool = catboost.Pool(data=train[feature_columns], label=train['y'], group_id=train['group'], cat_features=cat_features)

params = {
    'iterations': model.get_best_iteration(),
    'loss_function': 'YetiRank',
    'one_hot_max_size': 300,
}
model = catboost.CatBoost(params)
model.fit(train_pool)

0:	total: 20.1s	remaining: 5h 16m 26s
1:	total: 39s	remaining: 5h 7m 30s
2:	total: 57.4s	remaining: 5h 57s
3:	total: 1m 15s	remaining: 4h 58m 1s
4:	total: 1m 34s	remaining: 4h 56m 17s
5:	total: 1m 53s	remaining: 4h 55m 46s
6:	total: 2m 11s	remaining: 4h 54m 55s
7:	total: 2m 30s	remaining: 4h 53m 58s
8:	total: 2m 49s	remaining: 4h 53m 56s
9:	total: 3m 7s	remaining: 4h 53m 1s
10:	total: 3m 26s	remaining: 4h 52m 26s
11:	total: 3m 45s	remaining: 4h 52m 15s
12:	total: 4m 3s	remaining: 4h 51m 46s
13:	total: 4m 22s	remaining: 4h 51m 12s
14:	total: 4m 40s	remaining: 4h 50m 58s
15:	total: 4m 59s	remaining: 4h 50m 30s
16:	total: 5m 18s	remaining: 4h 49m 59s
17:	total: 5m 36s	remaining: 4h 49m 30s
18:	total: 5m 55s	remaining: 4h 49m 6s
19:	total: 6m 13s	remaining: 4h 48m 39s
20:	total: 6m 32s	remaining: 4h 48m 11s
21:	total: 6m 50s	remaining: 4h 47m 57s
22:	total: 7m 9s	remaining: 4h 47m 32s
23:	total: 7m 27s	remaining: 4h 47m 8s
33:	total: 10m 35s	remaining: 4h 44m 32s
34:	total: 10m 54s	remaini

In [ ]:
model.save_model('cat.txt')
# params = {
#     'loss_function': 'YetiRank',
#     'use_best_model': True,
#     'one_hot_max_size': 300,
# }

# model = catboost.CatBoost(params)
# model.load_model('cat.txt')

In [ ]:
all_users = users['user'].values
all_users

In [ ]:
# to avoid OOM
preds = []

n_split_prediction = 10
n_chunk = (len(all_users) + n_split_prediction - 1)// n_split_prediction
for i in range(0, len(all_users), n_chunk):
    target_users = all_users[i:i+n_chunk]

    candidates = create_candidates(transactions, target_users, 1)
    candidates = attach_features(transactions, users, items, candidates, 1)

    candidates['pred'] = model.predict(candidates[feature_columns])
    pred = candidates.groupby(['user', 'item'])['pred'].max().reset_index()
    pred = pred.sort_values(by=['user', 'pred'], ascending=False).reset_index(drop=True).groupby('user')['item'].apply(lambda x: list(x)[:12]).reset_index()
    preds.append(pred)

pred = pd.concat(preds).reset_index(drop=True)
assert len(pred) == len(all_users)
assert np.array_equal(pred['user'].values, all_users)

In [ ]:
for i in range(5):
    plot_images(dataset, pred['item'][i])

In [ ]:
mp_user = pd.read_pickle(f"input/{dataset}/mp_customer_id.pkl")
mp_item = pd.read_pickle(f"input/{dataset}/mp_article_id.pkl")

a_user = mp_user['val'].values
a_item = mp_item['val'].values

pred['customer_id'] = pred['user'].apply(lambda x: a_user[x])
pred['prediction'] = pred['item'].apply(lambda x: list(map(lambda y: a_item[y], x)))

pred['prediction'] = pred['prediction'].apply(lambda x: ' '.join(map(str, x)))

submission = pred[['customer_id', 'prediction']]
submission

In [ ]:
submission.to_csv('submission.csv', index=False)